<!-- 
Fixing the preview working directory
```{python}
import os
home = os.path.expanduser("~")
os.chdir(os.path.join(home,'eds-220-book'))
```
-->

# `geopandas`

GeoPandas is a Python library that extends the `pandas` library by adding support for geospatial data. In this lesson we will introduce the `geopandas` library to work with vector data. We will also make our first map. 

To begin with, let's import `geopandas` with its standard abbreviation `gpd`:

```{python}
# this is the library we will explore
import geopandas as gpd

# we will start using matplotlib for making maps
import matplotlib.pyplot as plt
```

## Data

In this lesson we will use simplified point data about wild pigs (*Sus scrofa*) sightings in California, USA from the [Global Biodiversity Information Facility](https://www.gbif.org). 

We can read in a shapefile with `geopandas` by using the `gpd.read_file()` function. 

```{python}
pigs = gpd.read_file('data/gbif_sus_scroga_california/gbif_sus_scroga_california.shp')
pigs.head()
```

::: {.callout-caution}
## One shapefile = multiple files
Although the parameter for `gpd.read_file()` is only the `.shp` file, remember that we need to have at least the `.shx` and `.dbf` files in the same directory as the `.shp` to read in the data. 
:::

## `GeoSeries` and `GeoDataFrame`

The core data structure in GeoPandas is the `geopandas.GeoDataFrame`. 
We can think of it as a `pandas.DataFrame` with a dedicated geometry column that can perform spatial operations. 

The **geometry column** in a `gpd.GeoDataFrame` holds the geometry (point, polygon, etc) of each spatial feature. Columns in the `gpd.GeoDataFrame` with attributes about the features are `pandas.Series` like in a regular `pd.DataFrame`. 

![Image source: GeoPandas documentation](/images/lesson-8-geopandas/dataframe.svg)

**Example**

First of all, notice that the leftmost column of `pigs` is a column named `geometry` whose values indicate points. 

```{python}
pigs.head(3)
```

As usual, we can check the type of our objects using the `type` Python function:

```{python}
# type of the pigs dataframe 
print(type(pigs))

# type of the geometry column
print(type(pigs.geometry))

# type of the gbifID column
print(type(pigs.gbifID))
```

The new data type of the geometry column is also reflected when we look at the data types of the columns in the data frame:

```{python}
pigs.dtypes
```

We can also check the type of each element in the geometry column using the `geom_type` attribute of a `gpd.GeoDataFrame`:

```{python}
pigs.geom_type
```

<!--

This is a `shapely` point. `shapely` is another Python library that to construct and modify points, lines, and polygons. We will talk more about it in the next lessons.
-->

## Geometric information

Two other important attributes of a `gpd.GeoDataFrame` are its coordinate reference system (CRS) and its extent. 

We can think of the **coordinate reference system (CRS)** as the instructions to locate each feature in our dataframe on the surface of the Earth. We access the CRS of a `gpd.GeoDataFrame` using the `crs` attribute:

```{python}
# access the CRS of the GeoDataFrame
pigs.crs
```

The **extent** of the geo-dataframe is the bounding box covering *all* the features in our geo-dataframe. This is formed by finding the points that are furthest west, east, south and north.

![Spatial extent of different vector data. Image Source: National Ecological Observatory Network (NEON)](/images/lesson-8-geopandas/spatial-extent.png)

We access the extent of a `gpd.GeoDataFrame` using the `total_bounds` attribute:

```{python}
pigs.total_bounds
```

## Data wrangling

GeoPandas is conveniently built on top of `pandas`, so we may use  everything we have learned about data selection, wrangling, and modification for a `pd.DataFrame`.

**Example**

Suppose we only want to use recent data for wild pig observations. A quick check shows that this dataframe has data since 1818:

```{python}
# use sort_index() method to order the index
pigs.year.value_counts().sort_index()
```

We can use our usual data selection to get data from 2020 onwards:

```{python}
# selet data from 2020 onwards
pigs_recent = pigs[pigs.year>=2020]

# print length of original dataframe
print(len(pigs))

# check length of new dataframe
len(pigs_recent)
```

## Plotting

### `plot()`

Similarly to a `pd.DataFrame`, a `gpd.GeoDataFrame` has a `plot()` method that we can call directly to create a quick view of our data. 
The geospatial information of the `gpd.GeoDataFrame` will be used to create the axes of the plot.

**Example**

This is a quick look at our recent pigs data:

```{python}
pigs_recent.plot()
```

### `matplotlib`'s `fig` and `ax`

Going forward, we will often want to make more complex visualizations where we add different layers to a graph and customize it. 
To do this we will use the `matplotlib` Python library for creating visualizations. 
We can interact with `matplotlib` via its `pyplot` interface, which we imported at the top of the notebook as 
```python
# import matplotlib with standard abbreviation
import matplotlib.pyplot as plt
```

Matplotlib graphs the data in a **figure** which can have one or more **axes**. 
The axis is only the area specified by the x-y axis and what is plotted in it. To create a new blank figure:

1. Initialize a new figure and axes by calling `pyplot`'s `subplots()` function, and
2. show the graph using `plt.show()`:

```{python}
# create a blank figure (fig) with an empty axis (ax)
fig, ax = plt.subplots()

# display figure
plt.show()
```

Notice we get a figure with a single empty axis. We can think of this step as setting a new blank canvas on which we will paint upon. 

::: {.callout-note}
## Functions with multiple return values
Notice that `plt.subplots()` is a function that returns two objects (has two outputs). 
:::

### Adding a layer

When using `matplotlib`, it can be useful to think of creating a plot as adding layers to an axis.
The general syntax to plot a datafram `df` onto an axis is:
```python
# create new figure and axis
fig, ax = plt.subplots()

# plot df on the ax axis
df.plot(ax=ax, 
        ...)   # other arguments for plot function

# display figure
plt.show()
``` 

**Example**

The first layer we want to add to our axis is the `pigs_recent` point data. 
We can plot our data using `matplotlib` like this:

```{python}
# create new figure and axis
fig, ax = plt.subplots()

# add pigs point plot to our figure's axis
pigs_recent.plot(ax=ax)

# display figure
plt.show
```

### Customization 

Matplotlib allows for a lot of customization. 
Some of it can be done directly in the `plot()` method for the dataframe (like we've done when ploting data using `pandas`), while other is done by updating attributes of the axis `ax`. 
The following image shows some examples of elements in the axis that can be updated.

![Image source: Matplotlib documentation](/images/lesson-8-geopandas/anatomy.png)

**Example**

Some basic customization for our pigs data could looke like this:

```{python}
# initialize empty figure
fig, ax = plt.subplots()

# add data to axis
# notice customization happens as arguments in plot()
pigs_recent.plot(ax=ax,
                 alpha=0.5,
                 color='brown'
                 )

# update axis 
# customization separate from the data plotting
ax.set_title('Reported "Sus scrofa" sightings in CA (2020-2023)')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# display figure
plt.show()
```

<!--
## Simple map

[California's government Open Data Portal](https://data.ca.gov/dataset/ca-geographic-boundaries)

```python
# initialize empty figure
fig, ax = plt.subplots()

# add CA polygon to axis
ca.plot(ax=ax)

# add data to axis
# notice customization happens as arguments in plot()
pigs_recent.plot(ax=ax,
                 alpha=0.5,
                 color='brown')

# update axis 
# customization separate from the data plotting
ax.set_title('Reported "Sus scrofa" sightings in CA (2020-2023)')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# display figure
plt.show()
```

-->

## References
GBIG data: GBIF.org (23 October 2023) GBIF Occurrence Download https://doi.org/10.15468/dl.qavhwp

[Geopandas Documentation - Introduction to GeoPandas](https://geopandas.org/en/stable/getting_started/introduction.html)

[Matplotlib Documentation - Basic Usage](
https://matplotlib.org/3.5.3/tutorials/introductory/usage.html#sphx-glr-tutorials-introductory-usage-py)

<!--
https://www.neonscience.org/resources/learning-hub/tutorials/intro-vector-data-r
-->